# Agentic Model: RATS AI Triage Classifier

Combat Triage AI - Complete Implementation with Quantization and SALT Protocol

Ready for Raspberry Pi Deployment

## Step 1: Setup

#### Step 1A: Installation

In [1]:
!pip install -U transformers
!pip install optimum[onnxruntime]
!pip install torch torchaudio
!pip install pyttsx3
!pip install sounddevice scipy

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers━━━━━━━━━━━━ 0/2 [tokenizers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [tokenizers]    WARNING: Ignoring invalid distribution ~ransformers (/home/aripontoni/CompSci 471/cs471-pex4/.venv/lib/python3.13/site-packages)
    Found existing installation: transformers 4.55.4━━━━━━━━━━━━━━ 1/2 [transformers]
    Uninstalling transformers-4.55.4:╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-4.55.4━━━━━━━━━━━━ 1/2 [trans

#### Step 1B: Imports

In [2]:
import os, torch, torchaudio, re, time, pyttsx3
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import write
from transformers import pipeline, WhisperForConditionalGeneration, AutoProcessor
from datetime import datetime

/home/aripontoni/CompSci 471/cs471-pex4/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Step 1C: Configuration

In [3]:
MODEL_ID = "openai/whisper-tiny.en"
DEVICE = "cpu"  # Pi doesn't have GPU

Audio Generation

In [4]:
def generate_triage_audio(result, output_dir="triage_audio"):
    """
    Generate audio file from triage results with robust voice handling
    
    Args:
        result: Dictionary containing triage assessment results
        output_dir: Directory to save audio files
        
    Returns:
        Path to generated audio or text file
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Build the spoken message
    message_parts = []
    
    # Triage category announcement
    category = result['triage_category']
    message_parts.append(f"Triage category: {category}.")
    
    # Add urgency based on category
    if category == "Immediate":
        message_parts.append("This patient requires immediate medical attention.")
    elif category == "Delayed":
        message_parts.append("This patient has serious injuries but can wait for treatment.")
    elif category == "Minimal":
        message_parts.append("This patient has minor injuries.")
    elif category == "Expectant":
        message_parts.append("This patient has injuries incompatible with life.")
    elif category == "Unknown":
        message_parts.append("Additional assessment data needed.")
    
    # Confidence level
    confidence_pct = int(result['confidence'] * 100)
    message_parts.append(f"Assessment confidence: {confidence_pct} percent.")
    
    # Key findings
    entities = result['entities']
    findings = []
    
    if entities.get('can_walk') is not None:
        findings.append(f"Patient {'can' if entities['can_walk'] else 'cannot'} walk")
    
    if entities.get('bleeding_severe'):
        findings.append("Severe bleeding detected")
    
    if entities.get('resp_rate') is not None:
        findings.append(f"Respiratory rate: {entities['resp_rate']} per minute")
    
    if entities.get('obeys_commands') is not None:
        findings.append(f"Patient {'obeys' if entities['obeys_commands'] else 'does not obey'} commands")
    
    if findings:
        message_parts.append("Key findings: " + ", ".join(findings) + ".")
    
    # Next question
    if result['next_question']:
        message_parts.append(f"Recommended next question: {result['next_question']}")
    
    # Combine all parts
    full_message = " ".join(message_parts)
    
    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    patient_id = result['patient_id'].replace('.mp3', '').replace(' ', '_')
    
    print(f"\n🔊 Generating audio output...")
    print(f"📝 Message: {full_message}")
    
    # Try to generate audio with pyttsx3
    try:
        import pyttsx3
        engine = pyttsx3.init()
        
        # Get available voices and use the first one
        voices = engine.getProperty('voices')
        if voices:
            # Use first available voice (usually the system default)
            engine.setProperty('voice', voices[0].id)
        
        # Set properties
        engine.setProperty('rate', 150)
        engine.setProperty('volume', 0.9)
        
        output_filename = f"{output_dir}/triage_{patient_id}_{timestamp}.wav"
        engine.save_to_file(full_message, output_filename)
        engine.runAndWait()
        
        print(f"✓ Audio saved to: {output_filename}")
        return output_filename
        
    except Exception as e:
        # Fallback: Save as text file
        print(f"⚠️  Audio generation failed: {e}")
        print(f"💾 Saving as text file instead...")
        
        output_filename = f"{output_dir}/triage_{patient_id}_{timestamp}.txt"
        with open(output_filename, 'w') as f:
            f.write(full_message)
        
        print(f"✓ Text saved to: {output_filename}")
        print(f"ℹ️  Audio generation requires proper system audio configuration")
        
        return output_filename

def generate_quick_alert_audio(category, output_dir="triage_audio"):
    """
    Generate quick alert audio with robust voice handling
    
    Args:
        category: Triage category
        output_dir: Directory to save audio files
        
    Returns:
        Path to generated audio or text file
    """
    os.makedirs(output_dir, exist_ok=True)
    
    alert_messages = {
        "Immediate": "Alert! Immediate medical attention required!",
        "Delayed": "Attention. Delayed category patient.",
        "Minimal": "Minimal injuries detected.",
        "Expectant": "Expectant category.",
        "Unknown": "Assessment incomplete."
    }
    
    message = alert_messages.get(category, "Triage assessment complete.")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

In [5]:
def record_audio_assessment(duration=10, sample_rate=16000, output_dir="recordings"):
    """
    Record audio from microphone for triage assessment
    
    Args:
        duration: Recording duration in seconds (default: 10)
        sample_rate: Audio sample rate (default: 16000 for Whisper)
        output_dir: Directory to save recordings
        
    Returns:
        Path to recorded audio file
    """
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\n🎤 RECORDING TRIAGE ASSESSMENT")
    print(f"{'='*60}")
    print(f"Duration: {duration} seconds")
    print(f"Speak clearly and include SALT assessment details:")
    print(f"  • Can the patient walk?")
    print(f"  • Is there severe bleeding?")
    print(f"  • What is the respiratory rate?")
    print(f"  • Does the patient obey commands?")
    print(f"  • Is there a radial pulse?")
    print(f"{'='*60}\n")
    
    # Countdown
    for i in range(3, 0, -1):
        print(f"Recording starts in {i}...")
        time.sleep(1)
    
    print("🔴 RECORDING NOW - Speak your assessment!")
    
    # Record audio
    recording = sd.rec(int(duration * sample_rate), 
                      samplerate=sample_rate, 
                      channels=1, 
                      dtype='int16')
    sd.wait()  # Wait until recording is finished
    
    print("✓ Recording complete!\n")
    
    # Save to file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{output_dir}/assessment_{timestamp}.wav"
    write(filename, sample_rate, recording)
    
    print(f"💾 Saved to: {filename}")
    
    return filename

def interactive_triage_session(sensor_data=None):
    """
    Run an interactive triage session with live recording
    
    Args:
        sensor_data: Optional dict of drone sensor readings
        
    Returns:
        Triage assessment result
    """
    print("\n" + "="*60)
    print("🚁 INTERACTIVE COMBAT TRIAGE SYSTEM")
    print("="*60)
    print("\nThis system will:")
    print("1. Record your verbal patient assessment")
    print("2. Transcribe and analyze the information")
    print("3. Apply SALT triage protocol")
    print("4. Provide triage category and next steps")
    print("\n" + "="*60)
    
    # Get recording duration from user
    try:
        duration = int(input("\nHow many seconds do you need? (default: 10): ") or "10")
    except ValueError:
        duration = 10
    
    # Record audio
    audio_file = record_audio_assessment(duration=duration)
    
    # Process with triage system
    print("\n🔄 Processing assessment...")
    result = triage_patient(audio_file, sensor_data=sensor_data, generate_audio=True)
    
    return result

def continuous_triage_mode(sensor_data=None):
    """
    Continuous triage mode - keeps asking for new assessments
    
    Args:
        sensor_data: Optional dict of drone sensor readings
    """
    print("\n" + "="*60)
    print("🔁 CONTINUOUS TRIAGE MODE")
    print("="*60)
    print("Press Ctrl+C to exit\n")
    
    patient_count = 0
    
    try:
        while True:
            patient_count += 1
            print(f"\n{'='*60}")
            print(f"PATIENT #{patient_count}")
            print(f"{'='*60}")
            
            # Run triage session
            result = interactive_triage_session(sensor_data=sensor_data)
            
            # Ask if user wants to continue
            continue_input = input("\n\nAssess another patient? (y/n): ").lower()
            if continue_input != 'y':
                break
                
    except KeyboardInterrupt:
        print("\n\n✓ Triage mode ended")
        print(f"Total patients assessed: {patient_count}")

## Step 2: Medical Tools

In [6]:
# Expanded combat medical vocabulary
COMBAT_MEDICAL_LEXICON = """
tourniquet, hemorrhage, massive hemorrhage, capillary refill, 
obey commands, airway patent, airway obstructed, 
respirations per minute, respiratory rate, breathing adequately,
radial pulse present, radial pulse absent, carotid pulse,
shock, hypotensive, pale, clammy, cold,
GSW, gunshot wound, blast injury, shrapnel, amputation,
conscious, unconscious, alert, verbal, pain, unresponsive,
chest seal, needle decompression, nasopharyngeal airway,
combat gauze, hemostatic agent, pressure dressing,
walking wounded, litter urgent, urgent surgical,
can walk, cannot walk, ambulatory, unable to walk
"""

## Step 3: Model Quantization

Useful for the rasberry pi on our drone 

In [7]:
# ========== MODEL LOADING WITH QUANTIZATION ==========
print("Loading and quantizing model...")

# Load model and processor separately
model = WhisperForConditionalGeneration.from_pretrained(MODEL_ID)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Quantize the model (makes it 4x smaller and faster)
quantized_model = torch.quantization.quantize_dynamic(
    model, 
    {torch.nn.Linear},  # Quantize linear layers
    dtype=torch.qint8   # Use 8-bit integers
)

# Create ASR pipeline with quantized model AND processor components
asr = pipeline(
    "automatic-speech-recognition",
    model=quantized_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=DEVICE,
    chunk_length_s=30,
    stride_length_s=5,
    return_timestamps=True
)

print(f"Model loaded and quantized successfully")
print(f"Device: {DEVICE}")

Loading and quantizing model...


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


Model loaded and quantized successfully
Device: cpu


### Step 3: Audio Processing

In [8]:
def preprocess_combat_audio(audio_path):
    """Handle noisy battlefield audio conditions"""
    wav, sr = torchaudio.load(audio_path)
    
    # Resample to 16kHz (Whisper requirement)
    if sr != 16000:
        wav = torchaudio.functional.resample(wav, sr, 16000)
    
    # Convert to mono if stereo
    if wav.shape[0] > 1:
        wav = torch.mean(wav, dim=0, keepdim=True)
    
    # Noise reduction - high-pass filter for wind/vehicle noise
    wav = torchaudio.functional.highpass_biquad(wav, 16000, cutoff_freq=200)
    
    # Normalize volume (gunfire may cause clipping)
    max_val = wav.abs().max()
    if max_val > 0:
        wav = wav / max_val
    
    return wav, 16000

In [9]:
def simple_vad_chunks(wav_path, min_speech_len=0.6):
    """Voice Activity Detection - remove silence"""
    wav, sr = torchaudio.load(wav_path)
    
    if sr != 16000:
        wav = torchaudio.functional.resample(wav, sr, 16000)
    
    # Simple energy-based VAD
    energy = wav.pow(2).mean(dim=0)
    threshold = energy.mean() * 0.3
    
    voiced_mask = energy > threshold
    if voiced_mask.sum() < 16000 * min_speech_len:
        return [wav_path]  # Too short, return original
    
    # For simplicity, return original if has enough speech
    return [wav_path]

In [10]:
# ========== TRANSCRIPTION ==========
def transcribe(path: str) -> dict:
    """Transcribe audio with medical vocabulary priming"""
    
    # Build prompt for medical context
    prompt_ids = processor.get_prompt_ids(text=COMBAT_MEDICAL_LEXICON)
    
    if isinstance(prompt_ids, np.ndarray):
        prompt_ids = prompt_ids.tolist()
    elif isinstance(prompt_ids, tuple):
        prompt_ids = list(prompt_ids)
    
    prompt_ids = torch.tensor(prompt_ids, dtype=torch.long, device=DEVICE)
    
    # Clear any old forced decoder IDs
    try:
        asr.model.generation_config.forced_decoder_ids = None
    except:
        pass
    
    result = asr(
        path,
        generate_kwargs={
            "prompt_ids": prompt_ids,
            "temperature": 0.0,
            "num_beams": 5,
            "do_sample": False,
        },
        return_timestamps=True
    )
    
    return result

def transcribe_with_vad(path):
    """Transcribe with voice activity detection"""
    out = {"text": "", "chunks": []}
    
    for chunk in simple_vad_chunks(path):
        r = transcribe(chunk)
        out["text"] += (" " + r["text"]).strip()
        if "chunks" in r:
            out["chunks"].extend(r["chunks"])
    
    return out

# ========== ENTITY EXTRACTION ==========
def extract_triage_entities(transcription_text):
    """Extract SALT-relevant medical information from transcription"""
    text_lower = transcription_text.lower()
    
    entities = {
        "can_walk": None,
        "bleeding_severe": False,
        "obeys_commands": None,
        "resp_rate": None,
        "radial_pulse": None,
        "mental_status": None,
        "cap_refill_sec": None
    }
    
    evidence = []
    
    
    # Walking ability
    walk_yes = ["can walk", "walking", "ambulatory", "able to walk"]
    walk_no = ["cannot walk", "can't walk", "unable to walk", "not walking"]
    
    for phrase in walk_yes:
        if phrase in text_lower:
            entities["can_walk"] = True
            evidence.append(f"Walking: '{phrase}' detected")
            break
    
    for phrase in walk_no:
        if phrase in text_lower:
            entities["can_walk"] = False
            evidence.append(f"Not walking: '{phrase}' detected")
            break
    
    # Severe bleeding
    bleeding_phrases = ["severe bleeding", "hemorrhage", "massive hemorrhage", 
                       "tourniquet applied", "massive bleeding", "heavy bleeding"]
    for phrase in bleeding_phrases:
        if phrase in text_lower:
            entities["bleeding_severe"] = True
            evidence.append(f"Severe bleeding: '{phrase}' detected")
            break
    
    # Command response
    obey_yes = ["obeys commands", "follows commands", "responsive to commands", "responding"]
    obey_no = ["does not obey", "doesn't obey", "unresponsive", "no response", 
               "not responding", "not obeying"]
    
    for phrase in obey_yes:
        if phrase in text_lower:
            entities["obeys_commands"] = True
            evidence.append(f"Obeys commands: '{phrase}' detected")
            break
    
    for phrase in obey_no:
        if phrase in text_lower:
            entities["obeys_commands"] = False
            evidence.append(f"Does not obey: '{phrase}' detected")
            break
    
    # Respiratory rate extraction
    resp_patterns = [
        r'(\d+)\s*breaths?\s*(?:per\s*minute)?',
        r'(\d+)\s*respirations?\s*(?:per\s*minute)?',
        r'respiratory\s*rate\s*(?:of\s*)?(\d+)',
        r'breathing\s*(?:at\s*)?(\d+)',
        r'(\d+)\s*rpm'
    ]
    
    for pattern in resp_patterns:
        match = re.search(pattern, text_lower)
        if match:
            entities["resp_rate"] = int(match.group(1))
            evidence.append(f"Respiratory rate: {match.group(1)} detected")
            break
    
    # Radial pulse
    pulse_yes = ["radial pulse present", "has radial pulse", "pulse present"]
    pulse_no = ["no radial pulse", "radial pulse absent", "no pulse"]
    
    for phrase in pulse_yes:
        if phrase in text_lower:
            entities["radial_pulse"] = True
            evidence.append(f"Radial pulse: '{phrase}' detected")
            break
    
    for phrase in pulse_no:
        if phrase in text_lower:
            entities["radial_pulse"] = False
            evidence.append(f"No radial pulse: '{phrase}' detected")
            break
    
    # Mental status
    if "alert" in text_lower:
        entities["mental_status"] = "alert"
        evidence.append("Mental status: alert")
    elif "verbal" in text_lower or "responds to verbal" in text_lower:
        entities["mental_status"] = "verbal"
        evidence.append("Mental status: verbal")
    elif "pain" in text_lower or "responds to pain" in text_lower:
        entities["mental_status"] = "pain"
        evidence.append("Mental status: pain")
    elif "unresponsive" in text_lower:
        entities["mental_status"] = "unresponsive"
        evidence.append("Mental status: unresponsive")
    
    return entities, evidence

## Step 5: Implement SALT Protocol

In [11]:
# ========== SALT TRIAGE RULES ==========
def salt_rules(entities, sensors=None):
    """
    Implement SALT (Sort, Assess, Lifesaving interventions, Treatment/Transport) triage
    
    Categories:
    - Immediate (Red): Life-threatening injuries, needs immediate care
    - Delayed (Yellow): Serious injuries, can wait for treatment
    - Minimal (Green): Minor injuries, walking wounded
    - Expectant (Black): Injuries incompatible with life
    """
    s = sensors or {}
    
    # Merge sensor data
    can_walk = entities.get("can_walk") or s.get("can_walk")
    severe_bleed = entities.get("bleeding_severe") or s.get("bleeding_detected")
    resp = entities.get("resp_rate") or s.get("resp_rate")
    obeys = entities.get("obeys_commands") or s.get("obeys_commands")
    radial_pulse = entities.get("radial_pulse") or s.get("radial_pulse")
    
    # SALT Algorithm
    # Step 1: Can the patient walk?
    if can_walk is True:
        return "Minimal"
    
    # Step 2: Assess for life-threatening bleeding
    if severe_bleed:
        return "Immediate"
    
    # Step 3: Check respirations
    if resp is None:
        return "Unknown"  # Need more data
    
    if resp == 0:
        return "Expectant"  # Not breathing
    
    if resp >= 30:
        return "Immediate"  # Respiratory distress
    
    # Step 4: Check mental status / obeys commands
    if obeys is False:
        return "Immediate"  # Altered mental status
    
    # Step 5: Check radial pulse (perfusion)
    if radial_pulse is False:
        return "Immediate"  # Poor perfusion
    
    # Default: injuries present but stable
    return "Delayed"

In [12]:
# ========== CONFIDENCE & NEXT QUESTION ==========
def calculate_confidence(entities):
    """Calculate confidence based on how much data we have"""
    total_fields = len(entities)
    filled_fields = sum(1 for v in entities.values() if v is not None and v is not False)
    return filled_fields / total_fields

def suggest_next_question(entities):
    """Ask medic for missing critical SALT info"""
    
    if entities["can_walk"] is None:
        return "Can the patient walk?"
    
    if not entities["bleeding_severe"] and entities.get("bleeding_severe") is None:
        return "Is there severe bleeding or hemorrhage?"
    
    if entities["resp_rate"] is None:
        return "What is the respiratory rate per minute?"
    
    if entities["obeys_commands"] is None:
        return "Does the patient obey commands?"
    
    if entities["radial_pulse"] is None:
        return "Is there a radial pulse present?"
    
    return None  # All critical data collected

# ========== SENSOR FUSION ==========
def fuse_sensor_data(audio_entities, drone_sensors):
    """Combine voice transcription with drone sensor data"""
    final_entities = audio_entities.copy()
    
    # Sensor data overrides uncertain voice data
    if drone_sensors.get("thermal_bleeding_detected") is not None:
        if audio_entities["bleeding_severe"] is None or not audio_entities["bleeding_severe"]:
            final_entities["bleeding_severe"] = drone_sensors["thermal_bleeding_detected"]
    
    if drone_sensors.get("movement_detected") is not None:
        if audio_entities["can_walk"] is None:
            final_entities["can_walk"] = drone_sensors["movement_detected"]
    
    if drone_sensors.get("heart_rate") is not None:
        # Estimate respiratory rate from heart rate if not available
        if audio_entities["resp_rate"] is None:
            # Rough estimate: normal resp is ~1/4 of heart rate
            final_entities["resp_rate"] = int(drone_sensors["heart_rate"] / 4)
    
    return final_entities



## Step 6: Test the model

In [13]:
def triage_patient(audio_path, sensor_data=None, generate_audio=True):
    """
    Complete combat triage pipeline with optional audio output
    
    Args:
        audio_path: Path to audio file of medic assessment
        sensor_data: Optional dict of drone sensor readings
        generate_audio: Whether to generate audio output (default: True)
        
    Returns:
        Full triage assessment with category and confidence
    """
    start_time = time.time()
    
    print(f"\n{'='*60}")
    print(f"TRIAGE ASSESSMENT INITIATED")
    print(f"{'='*60}\n")
    
    # Step 1: Transcribe audio
    print("📝 Transcribing audio...")
    transcription = transcribe_with_vad(audio_path)
    print(f"✓ Transcription: {transcription['text'][:100]}...")
    
    # Step 2: Extract medical entities
    print("\n🔍 Extracting medical information...")
    entities, evidence = extract_triage_entities(transcription["text"])
    
    # Step 3: Fuse with sensor data if available
    if sensor_data:
        print("🤖 Fusing with sensor data...")
        entities = fuse_sensor_data(entities, sensor_data)
    
    # Step 4: Apply SALT triage rules
    print("\n🏥 Applying SALT triage protocol...")
    triage_category = salt_rules(entities, sensor_data)
    
    # Step 5: Calculate confidence and suggest next question
    confidence = calculate_confidence(entities)
    next_question = suggest_next_question(entities)
    
    processing_time = time.time() - start_time
    
    # Format results
    result = {
        "patient_id": os.path.basename(audio_path),
        "transcription": transcription["text"],
        "entities": entities,
        "evidence": evidence,
        "triage_category": triage_category,
        "confidence": confidence,
        "next_question": next_question,
        "processing_time_sec": round(processing_time, 2),
        "timestamp": transcription.get("chunks", [])
    }
    
    # Print results
    print(f"\n{'='*60}")
    print(f"TRIAGE RESULTS")
    print(f"{'='*60}")
    print(f"🚑 Category: {triage_category}")
    print(f"📊 Confidence: {confidence*100:.0f}%")
    print(f"⏱️  Processing Time: {processing_time:.2f}s")
    print(f"\n📋 Extracted Information:")
    for key, value in entities.items():
        if value is not None:
            print(f"  • {key}: {value}")
    
    if next_question:
        print(f"\n❓ Recommended Question: {next_question}")
    
    if evidence:
        print(f"\n📝 Evidence:")
        for item in evidence:
            print(f"  • {item}")
    
    print(f"{'='*60}\n")
    
    # NEW: Generate audio output
    if generate_audio:
        audio_file = generate_triage_audio(result)
        result['audio_output'] = audio_file
        
        # Generate quick alert for immediate cases
        if triage_category == "Immediate":
            alert_file = generate_quick_alert_audio(triage_category)
            result['alert_audio'] = alert_file
    
    return result

In [15]:
# Test with audio generation

# ========== INTERACTIVE TESTING OPTIONS ==========

print("\n" + "="*60)
print("TRIAGE SYSTEM READY")
print("="*60)
print("\nChoose a mode:")
print("1. Single interactive assessment (with recording)")
print("2. Continuous triage mode (multiple patients)")
print("3. Test with existing audio file")
print("="*60)

mode = input("\nEnter mode (1/2/3): ").strip()

if mode == "1":
    # Single interactive session
    result = interactive_triage_session()
    
elif mode == "2":
    # Continuous mode
    continuous_triage_mode()
    
elif mode == "3":
    # Test with existing file
    AUDIO = input("Enter audio file path: ").strip() or "EnglishTriageTest 1.mp3"
    
    print("\nTEST 1: Audio transcription with audio output")
    result1 = triage_patient(AUDIO, generate_audio=True)
    
    print("\n\nTEST 2: Audio + sensor fusion with audio output")
    mock_sensor_data = {
        "thermal_bleeding_detected": False,
        "movement_detected": False,
        "heart_rate": 120
    }
    result2 = triage_patient(AUDIO, sensor_data=mock_sensor_data, generate_audio=True)
else:
    print("Invalid mode selected")


TRIAGE SYSTEM READY

Choose a mode:
1. Single interactive assessment (with recording)
2. Continuous triage mode (multiple patients)
3. Test with existing audio file
Invalid mode selected


## Step 7: Performance metrics of the model

In [16]:
# ========== PERFORMANCE METRICS ==========
print("\n" + "="*60)
print("MODEL PERFORMANCE METRICS")
print("="*60)
print(f"Model: {MODEL_ID}")
print(f"Quantized: Yes (8-bit)")
print(f"Device: {DEVICE}")
print(f"Model parameters: {sum(p.numel() for p in asr.model.parameters()) / 1e6:.1f}M")
print("="*60)


MODEL PERFORMANCE METRICS
Model: openai/whisper-tiny.en
Quantized: Yes (8-bit)
Device: cpu
Model parameters: 21.2M
